<img align="left" src="https://panoptes-uploads.zooniverse.org/project_avatar/86c23ca7-bbaa-4e84-8d8a-876819551431.png" type="image/png" height=100 width=100>
</img>
<h1 align="right">KSO Tutorials #4: Add new clips to a Zooniverse workflow</h1>
<h3 align="right">Written by @jannesgg and @vykanton</h3>
<h5 align="right">Last updated: Nov 8th, 2021</h5>

# Set up and requirements

### Import Python packages

In [ ]:
# Set the directory of the libraries
import sys
sys.path.append('..')

# Set to display dataframes as interactive tables
from itables import init_notebook_mode
init_notebook_mode(all_interactive=True)

# Import required modules
import utils.tutorials_utils as t_utils
import utils.server_utils as serv_utils
import utils.t4_utils as t4
import utils.zooniverse_utils as zoo

print("Packages loaded successfully")

### Choose your project

In [ ]:
project = t_utils.choose_project()

### Initiate SQL database and populate sites, movies and species

In [ ]:
# Specify the path of the movies 
movies_path = "/uploads"

# Specify the path of the sql database
db_path = "koster_lab.db"

# Initiate the SQL database 
%run -i "../db_starter/starter.py" --movies_path $movies_path --db_path $db_path --project_name $project.value

# Retrieve info about zooniverse clips

In [ ]:
# Save your Zooniverse user name and password.
zoo_user, zoo_pass = zoo.zoo_credentials()

In [ ]:
# Specify the Zooniverse information required throughout the tutorial
zoo_info = ["subjects"]

# Retrieve and store the Zooniverse information required throughout the tutorial in a dictionary
zoo_info_dict = zoo.retrieve_zoo_info(zoo_user, zoo_pass, project.value, zoo_info)

In [ ]:
# Populate the sql with subjects uploaded to Zooniverse
zoo.populate_subjects(zoo_info_dict["subjects"], project.value, db_path)

## List available clips to upload to Zooniverse

In [ ]:
import pandas as pd
import ipywidgets as widgets


def choose_project():
    
    # Specify location of the latest list of projects
    projects_csv = "../db_starter/projects_list.csv" 
    
    # Read the latest list of projects
    projects_df = pd.read_csv(projects_csv)
    
    # Display the project options
    choose_project = widgets.Dropdown(
        options=projects_df.Project_name.unique().tolist(),
        value=projects_df.Project_name.unique().tolist()[0],
        description="Project:",
        disabled=False,
    )
    
    display(choose_project)
    return choose_project

def choose_server():
    
    # Select server storage
    server_i = widgets.Dropdown(
        options=["local","AWS","Chalmers"],
        description='Choose server:',
        ensure_option=True,
        disabled=False,
    )
    
    display(server_i)
    
    return server_i    
    
def choose_single_workflow(workflows_df):

    layout = widgets.Layout(width="auto", height="40px")  # set width and height

    # Display the names of the workflows
    workflow_name = widgets.Dropdown(
        options=workflows_df.display_name.unique().tolist(),
        value=workflows_df.display_name.unique().tolist()[0],
        description="Workflow name:",
        disabled=False,
    )

    # Display the type of subjects
    subj_type = widgets.Dropdown(
        options=["frame", "clip"],
        value="clip",
        description="Subject type:",
        disabled=False,
    )

    display(workflow_name)
    display(subj_type)

    return workflow_name, subj_type

def choose_clip_workflows(workflows_df):

    layout = widgets.Layout(width="auto", height="40px")  # set width and height

    # Display the names of the workflows
    workflow_name = widgets.SelectMultiple(
        options=workflows_df.display_name.unique().tolist(),
        description="Workflow name:",
        disabled=False,
    )

    
    display(workflow_name)

    return workflow_name

In [ ]:
# Specify the server to connect to
#server_i = t_utils.choose_server()
server_i = choose_server()

In [ ]:
# Check available movies from the server to upload
movies_to_upload = server_i.value)

In [ ]:
# and exclude potential clips already uploaded (to avoid duplicates)

Make sure your workflows in Zooniverse have different names to avoid issues while selecting the workflow id

In [ ]:
# Select the subject_set to upload the files to

In [ ]:
# Specify how many clips per movie and/or site should be uploaded to Zooniverse

### Video modifications

In [ ]:
# Choose to reduce the size of the videos, remove audio or blur sensitive portions


### Preview of videos

### Preview of clips

In [ ]:
#

### Upload clips to Zooniverse

You may receive an error message related to file size if clips exceed the recommended limit for Zooniverse uploads. In this case, we recommend shortening the clip length to achieve a suitable filesize.

In [ ]:
# Upload clips to Zoo
